In [539]:
# Importing the relevant libraries for EDA and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
sns.set(style='white')
%matplotlib inline
from scipy import stats
plt.style.use("ggplot")

# Importing relevant libraries for machine learning modeling & confusion matrices
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import DataConversionWarning
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore')

In [540]:
# Loading the data
df = pd.read_csv('data/Terry_Stops_20240826.csv')

# Checking the first few rows of the data
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,36 - 45,-1,20160000398323,208373,Offense Report,NaN,4852,1953,M,Asian,...,15:18:00.0000000,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 2ND W - LINCOLN - PLATOON 1,N,N,North,L,L3
1,18 - 25,-1,20180000227180,559146,Citation / Infraction,NaN,5472,1964,M,Asian,...,00:07:00.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--TRAFFIC - BICYCLE VIOLATION,911,SOUTHWEST PCT - 3RD WATCH - F/W RELIEF,N,N,Southwest,F,F3
2,18 - 25,-1,20180000410091,498246,Offense Report,NaN,6081,1962,M,White,...,02:56:00.0000000,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 3RD W - BOY (JOHN) - PLATOON 1,N,N,North,B,B1
3,-,-1,20160000001637,146742,Field Contact,NaN,6924,1974,M,White,...,01:44:00.0000000,-,-,-,EAST PCT OPS - NIGHT ACT,N,N,East,C,C1
4,46 - 55,-1,20150000006037,104477,Field Contact,NaN,6732,1975,M,White,...,02:59:00.0000000,-,-,-,SOUTH PCT 3RD W - SAM - PLATOON 2,N,N,North,B,B2


In [541]:
# Checking the number of rows and columns of the data
df.shape

(61009, 23)

This data represents records of police reported stops under Terry v. Ohio, 392 U.S. 1 (1968). Each row represents a unique stop.

Each record contains perceived demographics of the subject, as reported by the officer making the stop and officer demographics as reported to the Seattle Police Department, for employment purposes.

This data contains 61009 entries with 23 columns. It is obtained from City of Seattle on https://data.seattle.gov/Public-Safety/Terry-Stops.

# Column Names and Descriptions
The following descriptions were provided by data.seattle.gov
This dataset contains the following data:

**Subject Age Group**: Subject Age Group (10 year increments) as reported by the officer.

**Subject ID**: Key, generated daily, identifying unique subjects in the dataset using a character to character match of first name and last name. "Null" values indicate an "anonymous" or "unidentified" subject. Subjects of a Terry Stop are not required to present identification.

**GO / SC Num**: General Offense or Street Check number, relating the Terry Stop to the parent report. This field may have a one to many relationship in the data.

**Terry Stop ID**: Key identifying unique Terry Stop reports.

**Stop Resolution**: Resolution of the stop as reported by the officer.

**Weapon Type**: Type of weapon, if any, identified during a search or frisk of the subject. Indicates "None" if no weapons was found.

**Officer ID**: Key identifying unique officers in the dataset.

**Officer YOB**: Year of birth, as reported by the officer.

**Officer Gender**: Gender of the officer, as reported by the officer.

**Officer Race**: Race of the officer, as reported by the officer.

**Subject Perceived Race**: Perceived race of the subject, as reported by the officer.

**Subject Perceived Gender**: Perceived gender of the subject, as reported by the officer.

**Reported Date**: Date the report was filed in the Records Management System (RMS). Not necessarily the date the stop occurred but generally within 1 day.

**Reported Time**: Time the stop was reported in the Records Management System (RMS). Not the time the stop occurred but generally within 10 hours.

**Initial Call Type**: Initial classification of the call as assigned by 911.

**Final Call Type**: Final classification of the call as assigned by the primary officer closing the event.

**Call Type**: How the call was received by the communication center.

**Officer Squad**: Functional squad assignment (not budget) of the officer as reported by the Data Analytics Platform (DAP).

**Arrest Flag**: Indicator of whether a "physical arrest" was made, of the subject, during the Terry Stop. Does not necessarily reflect a report of an arrest in the Records Management System (RMS).

**Frisk Flag**: Indicator of whether a "frisk" was conducted, by the officer, of the subject, during the Terry Stop.

**Precinct**: Precinct of the address associated with the underlying Computer Aided Dispatch (CAD) event. Not necessarily where the Terry Stop occurred.

**Sector**: Sector of the address associated with the underlying Computer Aided Dispatch (CAD) event. Not necessarily where the Terry Stop occurred.

**Beat**: Beat of the address associated with the underlying Computer Aided Dispatch (CAD) event. Not necessarily where the Terry Stop occurred.

In [542]:
# Getting a closer look at the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61009 entries, 0 to 61008
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         61009 non-null  object
 1   Subject ID                61009 non-null  int64 
 2   GO / SC Num               61009 non-null  int64 
 3   Terry Stop ID             61009 non-null  int64 
 4   Stop Resolution           61009 non-null  object
 5   Weapon Type               28444 non-null  object
 6   Officer ID                61009 non-null  object
 7   Officer YOB               61009 non-null  int64 
 8   Officer Gender            61009 non-null  object
 9   Officer Race              61009 non-null  object
 10  Subject Perceived Race    61009 non-null  object
 11  Subject Perceived Gender  61009 non-null  object
 12  Reported Date             61009 non-null  object
 13  Reported Time             61009 non-null  object
 14  Initial Call Type     

The data types of the columns are as follows:

## Column Classification
### Numerical Columns:
Subject ID,
GO / SC Num,
Terry Stop ID,
Officer YOB,

### Categorical Columns:
Subject Age Group,
Stop Resolution,
Weapon Type,
Officer ID,
Officer Gender,
Officer Race,
Subject Perceived Race,
Subject Perceived Gender,
Reported Date,
Reported Time,
Initial Call Type,
Final Call Type,
Call Type,
Officer Squad,
Arrest Flag,
Frisk Flag,
Precinct,
Sector,
Beat,

In [543]:
# Checking for missing values in the dataset
df.isnull().sum()

Subject Age Group               0
Subject ID                      0
GO / SC Num                     0
Terry Stop ID                   0
Stop Resolution                 0
Weapon Type                 32565
Officer ID                      0
Officer YOB                     0
Officer Gender                  0
Officer Race                    0
Subject Perceived Race          0
Subject Perceived Gender        0
Reported Date                   0
Reported Time                   0
Initial Call Type               0
Final Call Type                 0
Call Type                       0
Officer Squad                 561
Arrest Flag                     0
Frisk Flag                      0
Precinct                        0
Sector                          0
Beat                            0
dtype: int64

Here we can clearly see that the Weapon Type feature has a lot of values missing and also the Office Squad as well has 561 missing values. Still yet we need to check the dataset in depth to learn if there are any place holders, none or other unnecessary values/Characters.

In [544]:
# Creating a function that shows the value counts of each column in the dataset.
def col_values(df):
    """
    For use in Preprocessing and cleaning to find placeholder values
    Input: Data frame
    Output: Counts of unique values for each column
    """
    for col in df.columns:
        print(df[col].value_counts())
        print('-------------------------------------------------------')
        
col_values(df)

Subject Age Group
26 - 35         20373
36 - 45         13618
18 - 25         11573
46 - 55          7738
56 and Above     3221
1 - 17           2286
-                2200
Name: count, dtype: int64
-------------------------------------------------------
Subject ID
-1              35104
 7753260438        28
 7774286580        22
 7726918259        21
 7731717691        20
                ...  
 15606702593        1
 7735943699         1
 7738872582         1
 16724979306        1
 19137661313        1
Name: count, Length: 17000, dtype: int64
-------------------------------------------------------
GO / SC Num
20160000378750    16
20150000190790    16
20180000134604    14
20210000267148    14
20230000049052    14
                  ..
20150000006142     1
20180000000272     1
20200000339446     1
20220000283906     1
20220000018102     1
Name: count, Length: 48845, dtype: int64
-------------------------------------------------------
Terry Stop ID
32633045284    3
19324329995    3
19268585

In [545]:
# For ease of use let us rename the columns
df.columns = ['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat']

df.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat'],
      dtype='object')

In [546]:
df.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
0,36 - 45,-1,20160000398323,208373,Offense Report,NaN,4852,1953,M,Asian,...,15:18:00.0000000,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 2ND W - LINCOLN - PLATOON 1,N,N,North,L,L3
1,18 - 25,-1,20180000227180,559146,Citation / Infraction,NaN,5472,1964,M,Asian,...,00:07:00.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--TRAFFIC - BICYCLE VIOLATION,911,SOUTHWEST PCT - 3RD WATCH - F/W RELIEF,N,N,Southwest,F,F3
2,18 - 25,-1,20180000410091,498246,Offense Report,NaN,6081,1962,M,White,...,02:56:00.0000000,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 3RD W - BOY (JOHN) - PLATOON 1,N,N,North,B,B1
3,-,-1,20160000001637,146742,Field Contact,NaN,6924,1974,M,White,...,01:44:00.0000000,-,-,-,EAST PCT OPS - NIGHT ACT,N,N,East,C,C1
4,46 - 55,-1,20150000006037,104477,Field Contact,NaN,6732,1975,M,White,...,02:59:00.0000000,-,-,-,SOUTH PCT 3RD W - SAM - PLATOON 2,N,N,North,B,B2


## Data cleaning
Let us clean the data before we proceed to processing. Let us start by replacing the dashes and place holders with the more workable values first. Then we will go on to the more complex cleaning process.

In [547]:
# Replacing the dashes with Unknown
df = df.replace('-', 'Unknown')
df.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
0,36 - 45,-1,20160000398323,208373,Offense Report,NaN,4852,1953,M,Asian,...,15:18:00.0000000,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 2ND W - LINCOLN - PLATOON 1,N,N,North,L,L3
1,18 - 25,-1,20180000227180,559146,Citation / Infraction,NaN,5472,1964,M,Asian,...,00:07:00.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--TRAFFIC - BICYCLE VIOLATION,911,SOUTHWEST PCT - 3RD WATCH - F/W RELIEF,N,N,Southwest,F,F3
2,18 - 25,-1,20180000410091,498246,Offense Report,NaN,6081,1962,M,White,...,02:56:00.0000000,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,NORTH PCT 3RD W - BOY (JOHN) - PLATOON 1,N,N,North,B,B1
3,Unknown,-1,20160000001637,146742,Field Contact,NaN,6924,1974,M,White,...,01:44:00.0000000,Unknown,Unknown,Unknown,EAST PCT OPS - NIGHT ACT,N,N,East,C,C1
4,46 - 55,-1,20150000006037,104477,Field Contact,NaN,6732,1975,M,White,...,02:59:00.0000000,Unknown,Unknown,Unknown,SOUTH PCT 3RD W - SAM - PLATOON 2,N,N,North,B,B2


Officer_gender has 30 'N' values. We cannot be sure if 'N' stands for 'Not Available', 'Not Disclosed', or even 'Non-Gender Binary'. Since it's such a small amount of data, we'll just drop it.

In [548]:
# Dropping the entries with 'N' values from the officer_gender column.
df = df[df['officer_gender']!= 'N']
df.officer_gender.value_counts()


officer_gender
M    54072
F     6907
Name: count, dtype: int64

Officer_squad also has some NAN values. Since this information is less relevant to this particular task, it is better to just drop the column.

In [549]:
# Dropping the officer_squad column and assigning the data to a copy data frame.
df_copy = df.drop('officer_squad', axis=1)
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60979 entries, 0 to 61008
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   subject_age_group         60979 non-null  object
 1   subject_id                60979 non-null  int64 
 2   go_sc_num                 60979 non-null  int64 
 3   terry_stop_id             60979 non-null  int64 
 4   stop_resolution           60979 non-null  object
 5   weapon_type               28419 non-null  object
 6   officer_id                60979 non-null  object
 7   officer_yob               60979 non-null  int64 
 8   officer_gender            60979 non-null  object
 9   officer_race              60979 non-null  object
 10  subject_perceived_race    60979 non-null  object
 11  subject_perceived_gender  60979 non-null  object
 12  reported_date             60979 non-null  object
 13  reported_time             60979 non-null  object
 14  initial_call_type         6

As we saw it above, there are some subject IDs that are repeated multiple times. This could be either duplicates or repeat offenders. So it is crucial to investigate that feature.

In [550]:
# Checking the subject_id column
df_copy['subject_id'].value_counts()

subject_id
-1              35095
 7753260438        28
 7774286580        22
 7726918259        21
 7731717691        20
                ...  
 15606702593        1
 7735943699         1
 7738872582         1
 16724979306        1
 19137661313        1
Name: count, Length: 16988, dtype: int64

In [551]:
# Let us replace those -1 values in 'Subject_ID' with 'unassigned'
df_copy['subject_id'] = df_copy['subject_id'].replace(-1, 'unassigned')
df_copy.subject_id.value_counts()

subject_id
unassigned     35095
7753260438        28
7774286580        22
7726918259        21
7731717691        20
               ...  
15606702593        1
7735943699         1
7738872582         1
16724979306        1
19137661313        1
Name: count, Length: 16988, dtype: int64

Here it looks like we have multiple duplicates in 'Subject_IDs'. If that is the case this could make our dataset biased so we need to check closely to decide whether we have duplicates or not. We can do this by checking a number of columns namely 'subject_id', 'terry_stop_id' and 'officer_id'.

In [552]:
# Group by 'subject_id', 'terry_stop_id', and 'officer_id' and count occurrences
df_copy['count'] = df_copy.groupby(['subject_id', 'terry_stop_id', 'officer_id'])['subject_id'].transform('count')

# Create 'repeat_offenders' column based on count
df_copy['repeat_offenders'] = df_copy['count'].apply(lambda x: 'Yes' if x > 1 else 'No')

# Drop the 'count' column as it is no longer needed
df_copy = df_copy.drop(columns=['count'])

df_copy['repeat_offenders'].value_counts()

repeat_offenders
No     60784
Yes      195
Name: count, dtype: int64

This tells us we have 195 duplicated in our dataset but still we need to dig deeper before we conclusively decide.

**Terry Stop ID** also has some duplicate values worth checking.

In [553]:
# Checking terry stop id value counts
df_copy['terry_stop_id'].value_counts()

terry_stop_id
19324329995    3
19268585233    3
27511831225    3
36014210659    3
32633045284    3
              ..
87443          1
108886         1
274766         1
12093615563    1
31342435997    1
Name: count, Length: 60877, dtype: int64

In [554]:
# Listing the duplicates
dup_ids = df_copy[df_copy['terry_stop_id'].duplicated(keep=False)].sort_values(by = 'terry_stop_id')
# dup_ids = dup_ids[['subject_age_group', 'subject_id', 'go_sc_num', 
#                    'terry_stop_id', 'stop_resolution', 'weapon_type',
#                    'officer_id', 'reported_date', 'reported_time',
#                    'initial_call_type', 'final_call_type', 'arrest_flag',
#                    'frisk_flag', ]]
dup_ids

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
52,26 - 35,7810387129,20190000254490,8611673538,Field Contact,Knife/Cutting/Stabbing Instrument,7712,1987,M,White,...,01:09:47.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,Y,Unknown,Unknown,Unknown,Yes
43012,26 - 35,7810387129,20190000254490,8611673538,Field Contact,Blunt Object/Striking Implement,7712,1987,M,White,...,01:09:47.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,Y,Unknown,Unknown,Unknown,Yes
14583,26 - 35,7730805128,20190000268604,8677596250,Offense Report,Taser/Stun Gun,5630,1964,M,Black or African American,...,10:27:54.0000000,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--THEFT - CAR PROWL,911,N,Y,Southwest,F,F2,Yes
8805,26 - 35,7730805128,20190000268604,8677596250,Offense Report,Knife/Cutting/Stabbing Instrument,5630,1964,M,Black or African American,...,10:27:54.0000000,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--THEFT - CAR PROWL,911,N,Y,Southwest,F,F2,Yes
19773,18 - 25,9458419522,20190000285750,9585545373,Field Contact,Handgun,8382,1993,M,White,...,22:50:59.0000000,ASLT - PERSON SHOT OR SHOT AT,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,N,Y,East,E,E3,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20234,26 - 35,53848066671,20240000133855,56110860878,Arrest,Knife/Cutting/Stabbing Instrument,8846,1997,M,Black or African American,...,11:14:27.0000000,DISTURBANCE,"--ASSAULTS, OTHER",911,Y,Y,North,B,B1,Yes
46922,36 - 45,57754429915,20240000198136,57754515446,Arrest,Knife/Cutting/Stabbing Instrument,8584,1982,M,White,...,11:15:20.0000000,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--BURGLARY - NON RESIDENTIAL/COMMERCIAL,"TELEPHONE OTHER, NOT 911",Y,Y,South,O,O1,Yes
42736,36 - 45,57754429915,20240000198136,57754515446,Arrest,Blunt Object/Striking Implement,8584,1982,M,White,...,11:15:20.0000000,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--BURGLARY - NON RESIDENTIAL/COMMERCIAL,"TELEPHONE OTHER, NOT 911",Y,Y,South,O,O1,Yes
9130,18 - 25,7741755512,20240000210107,57961741719,Arrest,Taser/Stun Gun,8897,1987,M,White,...,05:21:37.0000000,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--BURGLARY - NON RESIDENTIAL/COMMERCIAL,911,Y,Y,East,G,G1,Yes


This may look like a confirmation of duplicate entries at first glance. However if we look carefully we can see that all these incidents have different weapon types even though they have identical subject, stop and officer ids. From this we can understand that these incidents are entries done by the same officer at the same time with the same subject who happen to be with multiple weapon types. After all it is a normal procedure for officers to place multiple entries of the same subject based on each weapon type found with.

we can verify that by running the same code we used above to check for duplicates only this time we will add the 'weapon_type' feature as well.

In [555]:
# Group by 'subject_id', 'terry_stop_id', and 'officer_id' and count occurrences
df_copy['count'] = df_copy.groupby(['subject_id', 'terry_stop_id', 'officer_id', 'weapon_type'])['subject_id'].transform('count')

# Create 'repeat_offenders' column based on count
df_copy['repeat_offenders'] = df_copy['count'].apply(lambda x: 'Yes' if x > 1 else 'No')

# Drop the 'count' column as it is no longer needed
df_copy = df_copy.drop(columns=['count'])

df_copy['repeat_offenders'].value_counts()

repeat_offenders
No    60979
Name: count, dtype: int64

In [556]:
df_copy.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
0,36 - 45,unassigned,20160000398323,208373,Offense Report,NaN,4852,1953,M,Asian,...,15:18:00.0000000,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,L,L3,No
1,18 - 25,unassigned,20180000227180,559146,Citation / Infraction,NaN,5472,1964,M,Asian,...,00:07:00.0000000,"SUSPICIOUS PERSON, VEHICLE, OR INCIDENT",--TRAFFIC - BICYCLE VIOLATION,911,N,N,Southwest,F,F3,No
2,18 - 25,unassigned,20180000410091,498246,Offense Report,NaN,6081,1962,M,White,...,02:56:00.0000000,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,B,B1,No
3,Unknown,unassigned,20160000001637,146742,Field Contact,NaN,6924,1974,M,White,...,01:44:00.0000000,Unknown,Unknown,Unknown,N,N,East,C,C1,No
4,46 - 55,unassigned,20150000006037,104477,Field Contact,NaN,6732,1975,M,White,...,02:59:00.0000000,Unknown,Unknown,Unknown,N,N,North,B,B2,No


This confirms our observation. Therefore since having this multiple entries of the same subjects can bloat our dataset and since the incidents are only 195, it is better to drop the duplicates and keep only the first entries.

In [557]:
# Dropping the duplicates and keeping the first instance
df_copy = df_copy.drop_duplicates('terry_stop_id', keep='first').sort_values(by = 'terry_stop_id')
df_copy

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
5536,1 - 17,unassigned,20150000084533,28020,Referred for Prosecution,Lethal Cutting Instrument,4585,1955,M,Hispanic or Latino,...,16:10:00.0000000,Unknown,Unknown,Unknown,N,Y,East,G,G2,No
34249,36 - 45,unassigned,20150000001428,28092,Field Contact,NaN,7634,1977,M,White,...,05:49:00.0000000,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No
22176,18 - 25,unassigned,20150000001428,28093,Field Contact,NaN,7634,1977,M,White,...,05:55:00.0000000,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No
46800,26 - 35,unassigned,20150000001437,28381,Field Contact,NaN,7634,1977,M,White,...,10:38:00.0000000,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No
35165,36 - 45,unassigned,20150000087329,28462,Offense Report,NaN,7634,1977,M,White,...,11:46:00.0000000,SUICIDE - CRITICAL,--CRISIS COMPLAINT - GENERAL,"TELEPHONE OTHER, NOT 911",N,Y,East,E,E3,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8952,18 - 25,33970989734,20240000173704,58505996345,Field Contact,Handgun,6848,1978,M,White,...,09:44:14.0000000,Unknown,Unknown,Unknown,N,Y,West,K,K3,No
43019,1 - 17,7734651568,20240000173704,58506837543,Field Contact,Unknown,6848,1978,M,White,...,11:33:58.0000000,Unknown,Unknown,Unknown,N,Y,West,K,K3,No
31174,1 - 17,53252554865,20240000173704,58506902793,Field Contact,Unknown,6848,1978,M,White,...,11:43:37.0000000,Unknown,Unknown,Unknown,N,Y,West,K,K3,No
32618,26 - 35,7729016487,20240000238040,58508727659,Field Contact,Unknown,8564,1987,M,Hispanic or Latino,...,15:55:53.0000000,ROBBERY - CRITICAL (INCLUDES STRONG ARM),--ROBBERY - STRONG ARM,911,N,Y,North,B,B1,No


Next let us check some repetitions in the general offense street check column.

In [558]:
# Investigating the repeated values in the go_sc_num column
stops = df_copy[df_copy['go_sc_num'] > 1]
stops['go_sc_num'].value_counts()

go_sc_num
20160000378750    16
20150000190790    16
20180000134604    14
20230000049052    14
20210000267148    14
                  ..
20180000178900     1
20180000177811     1
20180000178117     1
20180000177172     1
20240000238282     1
Name: count, Length: 48826, dtype: int64

In [559]:
# Looking closely
stops = stops[stops['go_sc_num'] == 20160000378750]
stops

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
36631,36 - 45,unassigned,20160000378750,208299,Offense Report,NaN,7492,1983,M,White,...,22:18:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
13031,26 - 35,unassigned,20160000378750,208300,Offense Report,NaN,7492,1983,M,White,...,22:22:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
41484,26 - 35,unassigned,20160000378750,208301,Offense Report,NaN,7492,1983,M,White,...,22:24:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
49658,18 - 25,unassigned,20160000378750,208302,Offense Report,NaN,7492,1983,M,White,...,22:31:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
48555,26 - 35,unassigned,20160000378750,208303,Offense Report,NaN,7492,1983,M,White,...,22:35:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
36586,46 - 55,unassigned,20160000378750,208304,Offense Report,NaN,7492,1983,M,White,...,22:39:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
58626,36 - 45,unassigned,20160000378750,208305,Offense Report,NaN,7492,1983,M,White,...,22:41:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
5940,46 - 55,unassigned,20160000378750,208306,Offense Report,NaN,7492,1983,M,White,...,22:45:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
39754,26 - 35,unassigned,20160000378750,208307,Offense Report,NaN,7492,1983,M,White,...,22:48:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No
55394,36 - 45,unassigned,20160000378750,208308,Offense Report,NaN,7492,1983,M,White,...,22:51:00.0000000,Unknown,Unknown,Unknown,N,Y,North,N,N3,No


Taking into account the dates, the separate Terry Stop ID's, the different Stop Resolutions and it all roughly happening within the same hour, it appears that this was a **dispute** of some sort in which an officer **collected Offense Reports from 12 people** and issued out **tickets 4 people** (because there was **no physical arrest** denoted by the column 'arrest_flag', these were **non-custodial** arrests/citations).  

Looking back at the Column Description document, the GO/SC Number is considered the **"parent report"** that contain **associated Terry Stops**. This confirms our observations.

## Report Date
Ok so now lets remove the timestamp from date, create a new columns "incident year" and "incident month" with the year and month of the incidents and drop the reported date.

In [560]:
# Checking the column reported date
df_copy.reported_date.dtype, df_copy.reported_date.head()

(dtype('O'),
 5536     2015-03-15T00:00:00Z
 34249    2015-03-16T00:00:00Z
 22176    2015-03-16T00:00:00Z
 46800    2015-03-17T00:00:00Z
 35165    2015-03-17T00:00:00Z
 Name: reported_date, dtype: object)

In [561]:
# Converting to date time format
df_copy['reported_date'] = pd.to_datetime(df_copy['reported_date'])

# Creating a new column with the year of the incident
df_copy['incident_year'] = df_copy['reported_date'].dt.year

# Creating a new column with the month of the incident
df_copy['incident_month'] = df_copy['reported_date'].dt.month

# Dropping the reported date column
df_copy = df_copy.drop('reported_date', axis=1)
df_copy.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders,incident_year,incident_month
5536,1 - 17,unassigned,20150000084533,28020,Referred for Prosecution,Lethal Cutting Instrument,4585,1955,M,Hispanic or Latino,...,Unknown,Unknown,N,Y,East,G,G2,No,2015,3
34249,36 - 45,unassigned,20150000001428,28092,Field Contact,NaN,7634,1977,M,White,...,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No,2015,3
22176,18 - 25,unassigned,20150000001428,28093,Field Contact,NaN,7634,1977,M,White,...,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No,2015,3
46800,26 - 35,unassigned,20150000001437,28381,Field Contact,NaN,7634,1977,M,White,...,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,No,2015,3
35165,36 - 45,unassigned,20150000087329,28462,Offense Report,NaN,7634,1977,M,White,...,--CRISIS COMPLAINT - GENERAL,"TELEPHONE OTHER, NOT 911",N,Y,East,E,E3,No,2015,3


## Officer Age
Let us create a new column 'officer_age' that holds the age value of the officer at the time of the incident. We can do this by subtracting the officer year of birth from the incident year.

In [562]:
# Creating a column that holds the officer's year.
df_copy['officer_age'] = df_copy['incident_year'] - df_copy['officer_yob']
df_copy.officer_age.unique()

array([ 60,  38,  29,  24,  33,  30,  31,  42,  50,  58,  54,  32,  34,
        28,  26,  39,  48,  44,  52,  25,  41,  59,  45,  35,  40,  49,
        23,  37,  46,  27,  56,  36,  47,  57,  43,  53,  61,  51,  55,
        67,  63,  62,  66,  22,  69,  65,  64,  70,  21,  68, 118, 119,
       120, 121,  71])

Wow we have some entries for officer age that are unrealistic. Let us fix that.

In [563]:
# Dropping unrealistic ages from the officers age column
df_copy = df_copy[df_copy['officer_age'] <= 100]

# Confirming our change
df_copy['officer_age'].describe()

count    60807.000000
mean        34.488102
std          8.267055
min         21.000000
25%         28.000000
50%         33.000000
75%         39.000000
max         71.000000
Name: officer_age, dtype: float64

So now our officer age looks more realistic ranging from 21 years to 71 years old. Let us now drop the officer year of birth column from the dataframe.

In [564]:
# Dropping the officer_yob column
df_copy = df_copy.drop('officer_yob', axis=1)
df_copy.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_gender',
       'officer_race', 'subject_perceived_race', 'subject_perceived_gender',
       'reported_time', 'initial_call_type', 'final_call_type', 'call_type',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat',
       'repeat_offenders', 'incident_year', 'incident_month', 'officer_age'],
      dtype='object')

Let us now proceed to the stop resolution. From common knowledge, we know that any arrest which is not flagged as one in the appropriate column is considered a "non-custodial arrest" or an instance where a citation was issued. 

In [565]:
# Checking the stop resolution
df_copy['stop_resolution'].value_counts()

stop_resolution
Field Contact               29439
Offense Report              15701
Arrest                      14722
Referred for Prosecution      728
Citation / Infraction         217
Name: count, dtype: int64

Even though this column tells us what happened after the incident, the `Field Contact` and `Offense Report` values do give us insight as to why an officer may have initiated a stop. So let us create columns for these values and drop the stop resolution column.

In [566]:
# Creating field_contact column that contains 'y' and 'n' values
df_copy['field_contact'] = df_copy['stop_resolution'].str.contains('Field Contact')
df_copy['field_contact'] = df_copy['field_contact'].map({True: 'Y', False: 'N'})

# Creating offense_report column that contains 'y' and 'n' values
df_copy['offense_report'] = df_copy['stop_resolution'].str.contains('Offense Report')
df_copy['offense_report'] = df_copy['offense_report'].map({True: 'Y', False: 'N'})

df_copy.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_gender,officer_race,subject_perceived_race,...,frisk_flag,precinct,sector,beat,repeat_offenders,incident_year,incident_month,officer_age,field_contact,offense_report
5536,1 - 17,unassigned,20150000084533,28020,Referred for Prosecution,Lethal Cutting Instrument,4585,M,Hispanic or Latino,Black or African American,...,Y,East,G,G2,No,2015,3,60,N,N
34249,36 - 45,unassigned,20150000001428,28092,Field Contact,NaN,7634,M,White,Multi-Racial,...,N,Unknown,Unknown,Unknown,No,2015,3,38,Y,N
22176,18 - 25,unassigned,20150000001428,28093,Field Contact,NaN,7634,M,White,White,...,N,Unknown,Unknown,Unknown,No,2015,3,38,Y,N
46800,26 - 35,unassigned,20150000001437,28381,Field Contact,NaN,7634,M,White,White,...,N,Unknown,Unknown,Unknown,No,2015,3,38,Y,N
35165,36 - 45,unassigned,20150000087329,28462,Offense Report,NaN,7634,M,White,White,...,Y,East,E,E3,No,2015,3,38,N,Y


In [567]:
# Checking our new columns
df_copy.offense_report.value_counts(), df_copy.field_contact.value_counts()

(offense_report
 N    45106
 Y    15701
 Name: count, dtype: int64,
 field_contact
 N    31368
 Y    29439
 Name: count, dtype: int64)

The weapon type column contains a lot of redundant values. Let us clean it up and organize it well.

In [568]:
# Checking the weapon type column
df_copy['weapon_type'].value_counts()

weapon_type
Unknown                                 24493
Lethal Cutting Instrument                1482
Knife/Cutting/Stabbing Instrument        1289
Handgun                                   384
Blunt Object/Striking Implement           150
Firearm                                   102
Firearm Other                             100
Other Firearm                              73
Club, Blackjack, Brass Knuckles            49
Mace/Pepper Spray                          48
None/Not Applicable                        18
Firearm (unk type)                         15
Taser/Stun Gun                             14
Fire/Incendiary Device                     12
Rifle                                      10
Club                                        9
Shotgun                                     5
Personal Weapons (hands, feet, etc.)        2
Automatic Handgun                           2
Brass Knuckles                              1
Blackjack                                   1
Poison                

In [569]:
# Weapon type categories
none = ['None/Not Applicable']

knife = ['Lethal Cutting Instrument', 'Knife/Cutting/Stabbing Instrument']

blunt_obj = ['Club, Blackjack, Brass Knuckles', 'Club', 'Blackjack', 'Brass Knuckles']
firearm = ['Firearm Other', 'Firearm (unk type)', 'Other Firearm', 'Rifle', 
          'Shotgun', 'Automatic Handgun', 'Handgun']
other = ['Taser/Stun Gun', 'Mace/Pepper Spray', 'Fire/Incendiary Device', 'Poison', 'Personal Weapons (hands, feet, etc.)']

# Creating a function called replace_val that takes the source data, column name, old name and a new name
def replace_val(df, col, old_val, new_val):
    for i in range(len(df[col])):
        for j in range(len(old_val)):
            if df[col].iloc[i] == old_val[j]:
                df[col].iloc[i] = df[col].iloc[i].replace(old_val[j], new_val)


# Applying the function to replace weapon type values
# replacing none
replace_val(df_copy, 'weapon_type', none, 'None')

# replacing knife
replace_val(df_copy, 'weapon_type', knife, 'Knife/Stabbing Instrument')

# replacing blunt object
replace_val(df_copy, 'weapon_type', blunt_obj, 'Blunt Object/Striking Implement')

# replacing firearm
replace_val(df_copy, 'weapon_type', firearm, 'Firearm')

# other
replace_val(df_copy, 'weapon_type', other, 'Other')

df_copy['weapon_type'].value_counts()

weapon_type
Unknown                            24493
Knife/Stabbing Instrument           2771
Firearm                              691
Blunt Object/Striking Implement      210
Other                                 77
None                                  18
Name: count, dtype: int64

Let us tidy up the reported time column as well

In [570]:
# Converting the time column to datetime format and keeping only the hour assigning it to a new reported_hour column
df_copy['reported_time'] = pd.to_datetime(df_copy['reported_time'])
df_copy['reported_hour'] = df_copy['reported_time'].dt.hour
df_copy = df_copy.drop('reported_time', axis=1)
df_copy.reported_hour.head()

5536     16
34249     5
22176     5
46800    10
35165    11
Name: reported_hour, dtype: int32

Great the reported time now has been arranged by hour in 24 hour format.

In [571]:
# CHecking the copy dataframe so far
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60807 entries, 5536 to 12853
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   subject_age_group         60807 non-null  object
 1   subject_id                60807 non-null  object
 2   go_sc_num                 60807 non-null  int64 
 3   terry_stop_id             60807 non-null  int64 
 4   stop_resolution           60807 non-null  object
 5   weapon_type               28260 non-null  object
 6   officer_id                60807 non-null  object
 7   officer_gender            60807 non-null  object
 8   officer_race              60807 non-null  object
 9   subject_perceived_race    60807 non-null  object
 10  subject_perceived_gender  60807 non-null  object
 11  initial_call_type         60807 non-null  object
 12  final_call_type           60807 non-null  object
 13  call_type                 60807 non-null  object
 14  arrest_flag             

Moving on let us work with the call types column. There are 13000+ entries in this column with an 'unknown' values which indicates that these instances were not put in the CAD system. This creates a major problem since it gives as a stop instance for no identifiable reasons. It is therefore better to drop these instances as we cannot assume the call types.

In [572]:
# Checking the call types column
df_copy['initial_call_type'].value_counts()

initial_call_type
Unknown                                       13406
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW     4769
SUSPICIOUS PERSON, VEHICLE, OR INCIDENT        4200
DISTURBANCE                                    3060
ASLT - CRITICAL (NO SHOOTINGS)                 2702
                                              ...  
-ASSIGNED DUTY - STAKEOUT                         1
REQUEST TO WATCH                                  1
ESCAPE - PRISONER                                 1
ANIMAL BITE                                       1
EXPLOSION                                         1
Name: count, Length: 181, dtype: int64

ESCAPE - PRISONER, PHONE - OBSCENE OR NUISANCE PHONE CALLS, EXPLOSION, ORDER - ASSIST DV VIC W/SRVC OF COURT ORDER, ASSIGNED DUTY - STAKEOUT, TEXT MESSAGE and SCHEDULED EVENT (RECURRING) all of these have very small values, it is better to drop them as well. We are keeping the ones with the unknown values as they are so many and we assume that these instances could be meant for privacy reasons.

In [573]:
# Dropping the unknown call types and the ones with small values
df_copy = df_copy[(df_copy['call_type'] != 'ESCAPE - PRISONER')&
                  (df_copy['call_type'] != 'PHONE - OBSCENE OR NUISANCE PHONE CALLS')&
                  (df_copy['call_type'] != 'EXPLOSION')&
                  (df_copy['call_type'] != 'ORDER - ASSIST DV VIC W/SRVC OF COURT ORDER')&
                  (df_copy['call_type'] != 'ASSIGNED DUTY - STAKEOUT')&
                  (df_copy['call_type'] != 'SCHEDULED EVENT (RECURRING)')&
                  (df_copy['call_type'] != 'TEXT MESSAGE')]


df_copy['call_type'].value_counts()

call_type
911                              28734
ONVIEW                           14013
Unknown                          13406
TELEPHONE OTHER, NOT 911          4099
ALARM CALL (NOT POLICE ALARM)      525
Name: count, dtype: int64

Alright now that the initial call type is cleared let us proceed frisk flag.

In [574]:
# Checking our dataframe.
df_copy.shape

(60777, 26)

In [575]:
# Checking the frisk flag column
df_copy['frisk_flag'].value_counts()

#Dropping the frisk flag instances with unknown values only 478 instances.
df_copy = df_copy[df_copy['frisk_flag']!= 'Unknown']

df_copy['frisk_flag'].value_counts()


frisk_flag
N    45834
Y    14465
Name: count, dtype: int64

In [576]:
# Checking the arrest flag column for any missing values
df_copy['arrest_flag'].value_counts()

arrest_flag
N    53789
Y     6510
Name: count, dtype: int64

The arrest flag column seems clean with no missing values. Out of the total instances we have only 6510 arrests which make up to 10%.

**Officer Race** has some instances with unknown or not specified values. It is better to combine them together and tidying up the column.

In [577]:
# Checking the officer race column
df_copy['officer_race'].value_counts()

officer_race
White                            43220
Two or More Races                 4201
Hispanic or Latino                3986
Asian                             2876
Not Specified                     2828
Black or African American         2403
Nat Hawaiian/Oth Pac Islander      545
American Indian/Alaska Native      240
Name: count, dtype: int64

In [578]:
# Combining the Unknown values into the not specified values of the officer race column
replace_val(df_copy, 'officer_race', ['Unknown/Unspecified', 'Not Stated'], 'Not Specified')

df_copy.officer_race.value_counts()

officer_race
White                            43220
Two or More Races                 4201
Hispanic or Latino                3986
Asian                             2876
Not Specified                     2828
Black or African American         2403
Nat Hawaiian/Oth Pac Islander      545
American Indian/Alaska Native      240
Name: count, dtype: int64

**Subject Gender** same as officer race has unknown and unable to determine values that need to be combined together.

In [579]:
# Combining the Unknown values into the unable to determine values of the subject perceived gender column
Unknown = ['Unknown']
replace_val(df_copy,'subject_perceived_gender', Unknown, 'Unable to Determine')

df_copy.subject_perceived_gender.value_counts()

subject_perceived_gender
Male                                                         47640
Female                                                       12004
Unable to Determine                                            608
Gender Diverse (gender non-conforming and/or transgender)       45
MULTIPLE SUBJECTS                                                2
Name: count, dtype: int64

In [580]:
# Dropping the gender diverse and multiple subjects values as they are very few and can create problems
df_copy = df_copy[(df_copy['subject_perceived_gender']!= 'Gender Diverse (gender non-conforming and/or transgender)') &
                  (df_copy['subject_perceived_gender']!= 'MULTIPLE SUBJECTS')]

df_copy.subject_perceived_gender.value_counts()

subject_perceived_gender
Male                   47640
Female                 12004
Unable to Determine      608
Name: count, dtype: int64

## Precinct, Sector and Beat
These are location data which can be very important in this process as they determine the probability of one getting stopped. However they have some place holder values which need to be cleaned up.

In [581]:
# Checking the precinct data
df_copy['precinct'].value_counts()

precinct
West         16657
North        12693
Unknown      10661
East          8159
South         7290
Southwest     4673
OOJ             97
FK ERROR        22
Name: count, dtype: int64

In [582]:
# Let us check what the FK ERROR is
df_copy[df_copy['precinct'] == 'FK ERROR']

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_gender,officer_race,subject_perceived_race,...,precinct,sector,beat,repeat_offenders,incident_year,incident_month,officer_age,field_contact,offense_report,reported_hour
40088,46 - 55,7736599528,20190000196167,8258190629,Field Contact,Unknown,7773,M,White,White,...,FK ERROR,99,99,No,2019,5,41,Y,N,5
53464,26 - 35,7749300947,20190000215001,8317517528,Field Contact,Unknown,8509,M,White,Unknown,...,FK ERROR,99,99,No,2019,6,34,Y,N,8
38123,26 - 35,8333698983,20190000222535,8333750884,Field Contact,Unknown,7793,M,White,White,...,FK ERROR,99,99,No,2019,6,33,Y,N,16
51247,26 - 35,7728529496,20190000222535,8333754250,Field Contact,Unknown,7793,M,White,Unknown,...,FK ERROR,99,99,No,2019,6,33,Y,N,16
49974,1 - 17,8194784044,20190000224323,8335625151,Field Contact,Unknown,7590,M,Hispanic or Latino,Unknown,...,FK ERROR,99,99,No,2019,6,51,Y,N,21
50407,56 and Above,7728365691,20190000240737,8544232751,Offense Report,Unknown,8386,M,White,White,...,FK ERROR,99,99,No,2019,7,33,N,Y,0
41551,26 - 35,7726362993,20190000283674,9258189887,Field Contact,Unknown,7793,M,White,Unknown,...,FK ERROR,99,99,No,2019,8,33,Y,N,18
22615,36 - 45,7727091519,20190000325595,10042391872,Field Contact,Knife/Stabbing Instrument,6844,M,Not Specified,White,...,FK ERROR,99,99,No,2019,8,51,Y,N,18
18028,18 - 25,10392618417,20190000350728,10392612439,Field Contact,Unknown,7603,M,White,Unknown,...,FK ERROR,99,99,No,2019,9,42,Y,N,14
1996,56 and Above,7760748894,20190000369575,10569761986,Field Contact,Unknown,8599,M,Two or More Races,Unknown,...,FK ERROR,99,99,No,2019,10,30,Y,N,18


It looks very interesting the data we have here is between 2019 and 2022 half of them being in 2019. This suggests that the error is most likely due to system failure. Since this incidents really occurred we cannot ignore them and must clean them.

In [583]:
prec = ['FK ERROR', 'OOJ'] # OOJ stands for Obstruction of Justice
sect = ['99']
beats = ['99', '99', 'OOJ']

# precinct
replace_val(df=df_copy, col='precinct', old_val=prec, new_val='Unknown')
# sector
replace_val(df=df_copy, col='sector', old_val=sect, new_val='Unknown')
# beat
replace_val(df=df_copy, col='beat', old_val=beats, new_val='Unknown')

df_copy['precinct'].value_counts(), df_copy['sector'].value_counts(), df_copy['beat'].value_counts()

(precinct
 West         16657
 North        12693
 Unknown      10780
 East          8159
 South         7290
 Southwest     4673
 Name: count, dtype: int64,
 sector
 Unknown    10739
 K           5597
 M           5091
 E           4284
 N           3554
 D           3427
 F           2815
 R           2763
 B           2733
 Q           2538
 L           2466
 O           2326
 S           2200
 U           2199
 G           2084
 W           1856
 C           1791
 J           1739
 OOJ           50
 Name: count, dtype: int64,
 beat
 Unknown    10783
 K3          3209
 M3          2472
 E2          1789
 N3          1761
 E1          1439
 M2          1349
 D1          1346
 N2          1340
 K2          1303
 R2          1286
 D2          1278
 M1          1273
 Q3          1213
 F2          1159
 K1          1085
 E3          1054
 B2          1042
 B1          1018
 U2          1015
 O1           945
 S2           857
 L2           852
 F3           836
 F1           820
 L1     

## Final Check
Ok so let us now check what we have done with our dataset before proceeding further.

In [584]:
# Checking the copy dataframe we have cleaned up
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60252 entries, 5536 to 12853
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   subject_age_group         60252 non-null  object
 1   subject_id                60252 non-null  object
 2   go_sc_num                 60252 non-null  int64 
 3   terry_stop_id             60252 non-null  int64 
 4   stop_resolution           60252 non-null  object
 5   weapon_type               28175 non-null  object
 6   officer_id                60252 non-null  object
 7   officer_gender            60252 non-null  object
 8   officer_race              60252 non-null  object
 9   subject_perceived_race    60252 non-null  object
 10  subject_perceived_gender  60252 non-null  object
 11  initial_call_type         60252 non-null  object
 12  final_call_type           60252 non-null  object
 13  call_type                 60252 non-null  object
 14  arrest_flag             

In [585]:
# Checking each column individually
col_values(df_copy)

subject_age_group
26 - 35         20163
36 - 45         13474
18 - 25         11414
46 - 55          7651
56 and Above     3189
1 - 17           2263
Unknown          2098
Name: count, dtype: int64
-------------------------------------------------------
subject_id
unassigned     34599
7753260438        27
7774286580        22
7726918259        21
7731717691        20
               ...  
19936471829        1
7780272733         1
7727725553         1
7725526013         1
58509790601        1
Name: count, Length: 16918, dtype: int64
-------------------------------------------------------
go_sc_num
20150000190790    16
20160000378750    16
20230000049052    14
20210000267148    14
20180000134604    14
                  ..
20180000001768     1
20180000001770     1
20180000182262     1
20180000001772     1
20240000238282     1
Name: count, Length: 48424, dtype: int64
-------------------------------------------------------
terry_stop_id
28020          1
12723381959    1
12715450809    1
1271

In [586]:
# Fixing the format of subject age group values
replace_val(df_copy, 'subject_age_group', ['26 - 35'], '26_35')
replace_val(df_copy, 'subject_age_group', ['18 - 25'], '18_25')
replace_val(df_copy, 'subject_age_group', ['36 - 45'], '36_45')
replace_val(df_copy, 'subject_age_group', ['46 - 55'], '46_55')
replace_val(df_copy, 'subject_age_group', ['56 and Above'], '56_up')
replace_val(df_copy, 'subject_age_group', ['1 - 17'], '1_17')


df_copy['subject_age_group'].value_counts()

subject_age_group
26_35      20163
36_45      13474
18_25      11414
46_55       7651
56_up       3189
1_17        2263
Unknown     2098
Name: count, dtype: int64

With that we say we have cleaned up the dataset for EDA and modeling. 

# Feature Engineering

## Same Races
Here we'll make a binary column 'same_race' that displays as 1 if the officer and the subject were the same and 0 if they are different.

To accomplish this, we need to make sure that the categories in 'Officer_Race' and 'Subject_Perceived_Race' have the same values and make any necessary changes.

In [587]:
# Checking the values of both columns
races = df_copy[['officer_race', 'subject_perceived_race']]
col_values(races)

officer_race
White                            43191
Two or More Races                 4195
Hispanic or Latino                3982
Asian                             2872
Not Specified                     2827
Black or African American         2401
Nat Hawaiian/Oth Pac Islander      545
American Indian/Alaska Native      239
Name: count, dtype: int64
-------------------------------------------------------
subject_perceived_race
White                                        29548
Black or African American                    18119
Unknown                                       6108
Asian                                         2070
Hispanic                                      1666
American Indian or Alaska Native              1638
Multi-Racial                                   796
Native Hawaiian or Other Pacific Islander      158
Other                                          149
Name: count, dtype: int64
-------------------------------------------------------


Ok so we can see that we don't have the same values for both columns. The differences are (Hispanic or Latino, Hispanic), (American Indian/Alaska Native, American Indian or Alaska Native), (Two or More Races, Multi-Racial), (Nat Hawaiian/Oth Pac Islander, Other), and (Not Specified, Unknown). Lets sort this out.


In [588]:
# Aligning the column values
native = ['American Indian/Alaska Native', 'American Indian or Alaska Native']
multi = ['Two or More Races']
other = ['Nat Hawaiian/Oth Pac Islander', 'Native Hawaiian or Other Pacific Islander']
unknown = ['Unknown']
hispanic = ['Hispanic or Latino']

# native
replace_val(df_copy, 'officer_race', native, 'Native American')
replace_val(df_copy, 'subject_perceived_race', native, 'Native American')
# multi
replace_val(df_copy, 'officer_race', multi, 'Multi-Racial')
# other
replace_val(df_copy, 'officer_race', other, 'Other')
replace_val(df_copy, 'subject_perceived_race', other, 'Other')
# unknown
replace_val(df_copy, 'subject_perceived_race', unknown, 'Not Specified')
# hispanic
replace_val(df_copy, 'officer_race', hispanic, 'Hispanic')

df_copy.officer_race.unique() 

array(['Hispanic', 'White', 'Asian', 'Black or African American',
       'Native American', 'Not Specified', 'Other', 'Multi-Racial'],
      dtype=object)

In [589]:
df_copy.subject_perceived_race.unique()

array(['Black or African American', 'Multi-Racial', 'White',
       'Native American', 'Not Specified', 'Asian', 'Hispanic', 'Other'],
      dtype=object)

In [590]:
# Now that the values of the two fields are identical, let us create a new column same_race and populate it
df_copy['same_race'] = np.nan
for i in range(len(df_copy['officer_race'])):
    if df_copy['officer_race'].iloc[i] == df_copy['subject_perceived_race'].iloc[i]:
        df_copy['same_race'].iloc[i] = 'Y'
    else:
        df_copy['same_race'].iloc[i] = 'N'

df_copy['same_race'].value_counts()

same_race
N    37263
Y    22989
Name: count, dtype: int64

## Gender Race
Let us do the same thing with the officer gender and the subject gender. First let us make sure that the genders in both columns match.

In [593]:
# Matching both genders
male = ['Male']
female = ['Female']

replace_val(df_copy, 'subject_perceived_gender', male, 'M')
replace_val(df_copy, 'subject_perceived_gender', female, 'F')

# Now that the values of the two fields are identical, let us create a new column dif_gender and populate it
df_copy['same_gender'] = np.nan
for g in range(len(df_copy['officer_gender'])):
    if df_copy['officer_gender'].iloc[g] == df_copy['subject_perceived_gender'].iloc[g]:
        df_copy['same_gender'].iloc[g] = 'Y'
    else:
        df_copy['same_gender'].iloc[g] = 'N'

# df_copy['subject_perceived_gender']!= df_copy['officer_gender']

df_copy['same_gender'].value_counts()


same_gender
Y    43800
N    16452
Name: count, dtype: int64

In [ ]:
# Creating a new dataframe for EDA
df_clean = df_copy
df_clean.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_gender,officer_race,subject_perceived_race,...,repeat_offenders,incident_year,incident_month,officer_age,field_contact,offense_report,reported_hour,dif_race,dif_gender,dif_race_gender
5536,1_17,unassigned,20150000084533,28020,Referred for Prosecution,Knife/Stabbing Instrument,4585,M,Hispanic,Black or African American,...,No,2015,3,60,N,N,16,True,True,Y
34249,36_45,unassigned,20150000001428,28092,Field Contact,NaN,7634,M,White,Multi-Racial,...,No,2015,3,38,Y,N,5,True,False,Y
22176,18_25,unassigned,20150000001428,28093,Field Contact,NaN,7634,M,White,White,...,No,2015,3,38,Y,N,5,False,False,N
46800,26_35,unassigned,20150000001437,28381,Field Contact,NaN,7634,M,White,White,...,No,2015,3,38,Y,N,10,False,False,N
35165,36_45,unassigned,20150000087329,28462,Offense Report,NaN,7634,M,White,White,...,No,2015,3,38,N,Y,11,False,False,N


In [596]:
df_clean.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_gender',
       'officer_race', 'subject_perceived_race', 'subject_perceived_gender',
       'initial_call_type', 'final_call_type', 'call_type', 'arrest_flag',
       'frisk_flag', 'precinct', 'sector', 'beat', 'repeat_offenders',
       'incident_year', 'incident_month', 'officer_age', 'field_contact',
       'offense_report', 'reported_hour', 'dif_race', 'dif_gender',
       'dif_race_gender', 'same_race'],
      dtype='object')

## Exporting to CSV
We are done with cleaning, feature engineering and preprocessing the dataset. Let us export it to a new CSV file.

In [ ]:
# # Exporting to csv file
# df_clean.to_csv('data/clean_Terry_stops_data.csv', index=False)

# print('Data exported to clean_data.csv successfully.')